In [2]:
# The imports

from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os

In [4]:
load_dotenv(override=True)
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from openai import AsyncOpenAI
load_dotenv(override=True)

MODEL = 'llama-3.3-70b-versatile'
GROQ_BASE_URL = "https://api.groq.com/openai/v1"
groq_api_key = os.getenv("GROQ_API_KEY")  # Add your Groq API key to your .env file

# Point OpenAI client to Groq endpoint
groq_client = AsyncOpenAI(
    api_key=groq_api_key,
    base_url=GROQ_BASE_URL
)

groq_model = OpenAIChatCompletionsModel(model=MODEL, openai_client=groq_client)


### And now.. bring on the Agent with Tools!

In [ ]:
instructions = """
You are a helpful assistant that answers the user's questions related to Stripe. 
Use the provided tools to answer the questions and save them when asked.
"""

sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "@modelcontextprotocol/server-filesystem", "args": [sandbox_path]}
stripe_params = {"command": "@stripe/mcp", "args": ["--tools=all"]}
# playwright_params = {"command": "uvx","args": [ "mcp-server-browser"]}

async with MCPServerStdio(params=files_params, client_session_timeout_seconds=600) as mcp_server_files:
    async with MCPServerStdio(params=stripe_params, client_session_timeout_seconds=600) as mcp_server_stripe:
        agent = Agent(
            name="investigator", 
            instructions=instructions, 
            model=groq_model,
            mcp_servers=[mcp_server_files, mcp_server_stripe]
            )
        with trace("investigate"):
            result = await Runner.run(agent, "Get the recipe for banoffee, then summarize it in markdown to banoffee.md")
            print(result.final_output)


Error initializing MCP server: [Errno 2] No such file or directory: 'npx'


FileNotFoundError: [Errno 2] No such file or directory: 'npx'

### Check out the trace

https://platform.openai.com/traces

### Now take a look at some MCP marketplaces

https://mcp.so

https://glama.ai/mcp

https://smithery.ai/

https://huggingface.co/blog/LLMhacker/top-11-essential-mcp-libraries

HuggingFace great community article:
https://huggingface.co/blog/Kseniase/mcp



